# Generating meshes with Wildmeshing

In this demo, we will show how to generate a mesh from STL/ply files based on MRI images.
We use the files from the paper [In-silico molecular enrichment and clearance of the human intracranial space](datasets.md#data-from-the-paper-in-silico-molecular-enrichment-and-clearance-of-the-human-intracranial-space)

We start by importing all the packages required for this task.


In [ ]:
from mpi4py import MPI
import wildmeshing
import dolfinx
from pathlib import Path
import json
import numpy as np
import pyvista
import ufl
import basix.ufl
import os

Next, we use the already downloaded stl files from this chapter. We will use WildMeshing to combine these.
We create unique volume markers for the lateral ventricles (`LV.ply`) and the third and fourth ventricle (`V34.ply`). Next we embed the ventricular system within the brain parenchyma (`parenchyma_incl_ventr.ply`), before we add the dura interface (`skull.ply`), which is the interfacebetween the CSF and the skull.

We extract these files from the `"WILDFENICS_DATA_PATH"`. If you have not set this variable, please change it to the appropriate location on your system.


In [ ]:
folder = Path(os.environ["WILDFENICS_DATA_PATH"])
assert folder.exists(), "Could not find surface files"
tree = {
    "operation": "union",
    "left": str((folder / "skull.ply").absolute().as_posix()),
    "right": {
        "operation": "union",
        "left": str((folder / "parenchyma_incl_ventr.ply").absolute().as_posix()),
        "right": {
            "operation": "union",
            "left": str((folder / "LV.ply").absolute().as_posix()),
            "right": str((folder / "V34.ply").absolute().as_posix()),
        },
    },
}

{'operation': 'union', 'left': '/root/shared/stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/ventricles.final.stl', 'right': '/root/shared/stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/rhchoroid.plexus.final.stl'}
TBB threads 16
o-[I/O         ] Loading file /root/shared/stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/ventricles.final.stl...
                 (FP64) nb_v:134064 nb_e:0 nb_f:44688 nb_b:134064 tri:1 dim:3
                 Attributes on vertices: point[3]
                 Loading file /root/shared/stl_files/mhornkjol-mri2fem-ii-chapter-3-code-ff74dab/stl_files/rhchoroid.plexus.final.stl...
                 (FP64) nb_v:7908 nb_e:0 nb_f:2636 nb_b:7908 tri:1 dim:3
                 Attributes on vertices: point[3]
bbox_diag_length = 145.674
ideal_edge_length = 14.5674
stage = 2
eps_input = 0.145674
eps = 0.0806437
eps_simplification = 0.064515
eps_coplanar = 0.000145674
dd = 0.0971161
dd_simplification = 0.0776929


Next, we can set up a 3D meshing instance and load in the instruction-set


In [ ]:
tetra = wildmeshing.Tetrahedralizer(
    stop_quality=10, max_its=30, edge_length_r=0.03, epsilon=0.00225
)
tetra.load_csg_tree(json.dumps(tree))

Then, we can generate the mesh


In [3]:
# Create mesh
tetra.tetrahedralize()

[2025-02-17 08:58:13.832] [float-tetwild] [info] remove duplicates: 
[2025-02-17 08:58:13.832] [float-tetwild] [info] #v: 141972 -> 23655
[2025-02-17 08:58:13.832] [float-tetwild] [info] #f: 47324 -> 47324
collapsing 0.897446
swapping 0.015215
[2025-02-17 08:58:14.751] [float-tetwild] [info] remove duplicates: 
[2025-02-17 08:58:14.751] [float-tetwild] [info] #v: 6583 -> 6583
[2025-02-17 08:58:14.751] [float-tetwild] [info] #f: 13180 -> 13180
[2025-02-17 08:58:14.751] [float-tetwild] [info] #v = 6583
[2025-02-17 08:58:14.751] [float-tetwild] [info] #f = 13179
#boundary_e1 = 1
#boundary_e2 = 2
[2025-02-17 08:58:14.762] [float-tetwild] [info] preprocessing 0.960763s
[2025-02-17 08:58:14.762] [float-tetwild] [info] 
[2025-02-17 08:58:14.817] [float-tetwild] [info] #v = 8433
[2025-02-17 08:58:14.817] [float-tetwild] [info] #t = 50157
[2025-02-17 08:58:14.817] [float-tetwild] [info] tetrahedralizing 0.055105s
[2025-02-17 08:58:14.817] [float-tetwild] [info] 
[2025-02-17 08:58:14.817] [float

We want to use this mesh within FEniCSx, so we extract the cells, nodes and volume markers from the mesh instance in wildmeshing.


In [4]:
point_array, cell_array, marker = tetra.get_tet_mesh()

[2025-02-17 08:58:20.582] [float-tetwild] [info] after winding number
[2025-02-17 08:58:20.582] [float-tetwild] [info] #v = 5396
[2025-02-17 08:58:20.582] [float-tetwild] [info] #t = 15117
[2025-02-17 08:58:20.582] [float-tetwild] [info] winding number 1.73978e+09s
[2025-02-17 08:58:20.582] [float-tetwild] [info] 


We can pass these into the DOLFINx mesh constructor called `create_mesh`.


In [ ]:
mesh = dolfinx.mesh.create_mesh(
    MPI.COMM_WORLD,
    cell_array.astype(np.int64),
    point_array,
    ufl.Mesh(basix.ufl.element("Lagrange", "tetrahedron", 1, shape=(3,))),
)

We add the cell markers as `CellTags` with the following lines of code


In [ ]:
local_entities, local_values = dolfinx.io.gmshio.distribute_entity_data(
    mesh,
    mesh.topology.dim,
    cell_array.astype(np.int64),
    marker.flatten().astype(np.int32),
)
adj = dolfinx.graph.adjacencylist(local_entities)
ct = dolfinx.mesh.meshtags_from_entities(
    mesh,
    mesh.topology.dim,
    adj,
    local_values.astype(np.int32, copy=False),
)

We store the mesh and cell markers to file for usage in later tutorials


In [ ]:
with dolfinx.io.XDMFFile(mesh.comm, folder / "brain.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_meshtags(ct, mesh.geometry)

We visualize the resulting mesh and cell tags with pyvista


In [ ]:
pv_grid = pyvista.UnstructuredGrid(*dolfinx.plot.vtk_mesh(mesh))
pv_grid.cell_data["marker"] = ct.values

In [ ]:
pyvista.start_xvfb(1.0)
plotter = pyvista.Plotter()
plotter.add_mesh(pv_grid)
plotter.show()

We also create som slices to see the various markers within the mesh


In [ ]:
slices = pv_grid.slice_along_axis(n=7, axis="y")
slices.plot()